# Исследование надёжности заёмщиков

Цель проекта: определить есть ли зависимость между различными характеристиками заёмщиков (семейным положением, количеством детей, доходом и целью кредита) и возвращением кредита в срок.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd 
import numpy as np
data = pd.read_csv('/datasets/data.csv')
data.info() #общая информация о таблице
data.head() #пять первых строк таблицы

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Вывод**

В столбцах "days_employed" и "total_income" есть пропуски, столбец "days_employed" имеет некорректные значения (отрицательные или очень большие). В каких единицах представлен ежемесячный доход? Столбец "education" нужно привести к нижнему регистру

## Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
#заменяю пропуски в столбце со стажем на основе средних данных с учетом типа занятости
#data['days_employed'] = data['days_employed'].astype('int')
#data['days_employed'] = abs(data['days_employed']) #преобразую отрицательные значения в положительные

data.loc[(data['income_type'] == 'пенсионер') & (data['dob_years'] < 30)] #встречается
#много пенсионеров младше 30 лет
data['days_employed'] = data['days_employed'].fillna(value = ' ') #заменяю пропуски из-за описанной ниже пробелемы

for i in range(len(data)): #в цикле перебираю все значения из столбца со стажем
    if data.loc[i, 'days_employed'] != ' ': #из непропущенных значений считаю средний коэффициент
        
        if data.loc[i, 'dob_years'] > 50 and data.loc[i, 'income_type'] == 'пенсионер':
        #для пенсионеров отдельно, так как они какое-то время уже не работают
            x = data.loc[i, 'days_employed']
            y = data.loc[i, 'dob_years']
            k_old = abs(x) / (y - 18)
            k_old = k_old.mean()
    
        elif 30 > data.loc[i, 'dob_years'] > 18 and data.loc[i, 'income_type'] != 'пенсионер':
        #для людей младше 30, так как если их объединить со старшими, будет явно неправильный стаж
            x = data.loc[i, 'days_employed']
            y = data.loc[i, 'dob_years']
            k_young = abs(x) / (y - 18)
            k_young = k_young.mean()
            
        elif data.loc[i, 'dob_years'] >= 30 and data.loc[i, 'income_type'] != 'пенсионер':
        #для людей среднего возраста: не на пенсии, но старше 30
            x = data.loc[i, 'days_employed']
            y = data.loc[i, 'dob_years']
            k_else = abs(x) / (y - 18)
            k_else = k_else.mean()
            
    else: #считаю значения, чтобы заполнить пропуски
        
        if data.loc[i, 'income_type'] == 'пенсионер': #для пенсионеров
            data.loc[i, 'days_employed'] = k_old * (data.loc[i, 'dob_years'] - 18)
            
        else:
            
            if data.loc[i, 'dob_years'] >= 30: #для среднего возраста
                data.loc[i, 'days_employed'] = k_else * (data.loc[i, 'dob_years'] - 18)
                
            else: #для молодых
                data.loc[i, 'days_employed'] = k_young * (data.loc[i, 'dob_years'] - 18)
                
data.loc[data['days_employed'] == ' '] #проверяю, не осталось ли пропусков в столбце
            
#заменяю пропуски в столбце с заработком, но сначала убираю дубликаты в образовании
data['education'].value_counts() #дубликаты в разном регистре
data['education'] = data['education'].str.lower() #перевод в нижний регистр

data['total_income'] = data['total_income'].fillna(value = ' ')

for i in range(len(data)): #перебираю все значения в цикле

    if data.loc[i, 'total_income'] == ' ': #если есть пропуск, записываю значения в других столбцах
        ed = data.loc[i, 'education']
        ink = data.loc[i, 'income_type']
        fam = data.loc[i, 'family_status']
        gen = data.loc[i, 'gender']
        #ищу людей с такими же значениями в ключевых столбцах, для них считаю среднее по доходу
        data_mean = data.loc[(data['education'] == ed) & (data['income_type'] == ink) & (data['family_status'] == fam) & (data['gender'] == gen)]['total_income']
        data_mean = pd.to_numeric(data_mean, errors = 'coerce')
        mean = data_mean.mean()
        data.loc[i, 'total_income'] = mean #подставляю средний доход вместо пропуска
        
data.loc[data['total_income'] == ' ']
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null object
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21522 non-null object
purpose             21525 non-null object
dtypes: int64(5), object(7)
memory usage: 2.0+ MB


In [12]:
np.random.seed([3,1415])
df = pd.DataFrame(np.random.choice((1, np.nan), (10, 2)))
df

,0,1
0,1.0,NaN
1,1.0,NaN
2,1.0,1.0
3,NaN,1.0
4,NaN,NaN
5,NaN,NaN
6,1.0,NaN
7,NaN,1.0
8,NaN,1.0
9,1.0,1.0


In [13]:
df.isna().any(axis=1)

0     True
1     True
2    False
3     True
4     True
5     True
6     True
7     True
8     True
9    False
dtype: bool

In [14]:
df1 = df[df.isna().any(axis=1)]
df1

,0,1
0,1.0,NaN
1,1.0,NaN
3,NaN,1.0
4,NaN,NaN
5,NaN,NaN
6,1.0,NaN
7,NaN,1.0
8,NaN,1.0


**Вывод**

Я провела замену пропусков в двух столбцах.
В столбце со стажем работы я заменила их на средний показатель стажа в зависимости от группы занятости и возраста человека. Среди пенсионеров встретились люди младше 30 лет, их я не учитывала в расчете среднего. Во всех возрастных группах были люди с возрастом 0, их я тоже не учитывала при расчете. Для вычисления стажа я не стала брать среднее значение, так как он линейно зависит от возраста, по моему мнению.
Для вычисления зависимости стажа от возраста я написала уравнение x=k(y-18), где х - стаж в днях, y - возраст в годах, k - некоторый коэффициент, который я сначала нахожу для пенсионеров отдельно, для всех остальных отдельно. Потом возраст человека, стаж которого неизвестен, подставляю в это уравнение и нахожу стаж.

В столбце с доходом я расчитала средний доход для человека с пропуском как среднее из дохода людей, совпадающих с ним по четырём группам: образование, занятость, пол и семейное положение. Так как все эти характеристики влияют на уровень дохода.
Из-за сложного перебора значений для вычисления среднего эта часть работает очень медленно, но у меня нет идей по улучшению.

### Замена типа данных

In [3]:
data['days_employed'] = data['days_employed'].astype('int') #преобразовываю в целочисленный тип
data['days_employed'] = abs(data['days_employed']) #преобразую отрицательные значения в положительные
data.head()


data['income_type'].value_counts() #объединяю малочисленные группы
data.loc[data['income_type'] == 'предприниматель', 'income_type'] = 'сотрудник'
data.loc[data['income_type'] == 'в декрете', 'income_type'] = 'безработный'
data.loc[data['income_type'] == 'студент', 'income_type'] = 'безработный'
data['income_type'].value_counts() #проверка

days_sotr = data[data['income_type'] == 'сотрудник']['days_employed'].mean() #считаю средний стаж
days_komp = data[data['income_type'] == 'компаньон']['days_employed'].mean() #для разных групп
days_pens = data[data['income_type'] == 'пенсионер']['days_employed'].mean() #по типу занятости
days_goss = data[data['income_type'] == 'госслужащий']['days_employed'].mean()
#data['days_employed'] = data['days_employed'].fillna(value = ' ') #заменяю пропуски на среднее значение
data.loc[(data['days_employed'] == ' ') & (data['income_type'] == 'сотрудник'), 'days_employed'] = days_sotr
data.loc[(data['days_employed'] == ' ') & (data['income_type'] == 'компаньон'), 'days_employed'] = days_komp
data.loc[(data['days_employed'] == ' ') & (data['income_type'] == 'пенсионер'), 'days_employed'] = days_pens
data.loc[(data['days_employed'] == ' ') & (data['income_type'] == 'госслужащий'), 'days_employed'] = days_goss
data.loc[data['days_employed'] == ' ']

/opt/conda/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


**Вывод**

Дни являются целочисленными, поэтому я заменила их тип на integer. Хотя, их значения остались некорректными, у меня нет идей, как их исправить, и значит ли они что-то. Они не нужны для выводов по таблице, поэтому дальше преобразовывать я их не буду.
Данные по ежемесячнму доходу я оставила вещественными, так как доход вополне может быть не целочисленным

### Обработка дубликатов

**Вывод**

In [4]:
#проверка столбцов по очереди на наличие дубликатов
data['children'].value_counts() #смотрю, какие есть значения в столбце
data[data['children'] == 20] #проверяю подозрительные 20 и -1, исправляю как опечатки
data.loc[data['children'] == 20, 'children'] = 2
data.loc[data['children'] == -1, 'children'] = 1
print(data['children'].value_counts()) #проверка
print()

data['education'].value_counts() #дубликаты в разном регистре
data['education'] = data['education'].str.lower() #перевод в нижний регистр
print(data['education'].value_counts()) #проверка
print()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64



Были исправлены дубликаты в разных регистрах и опечатки. Причиной их появления считаю человеческий фактор. Так же я объединила малочисленные группы в столбцах 'income_type' и 'children'

### Лемматизация

In [5]:
from pymystem3 import Mystem
m = Mystem()
purpose = data['purpose'].unique() #создаю список из уникальных значений цели получения кредита
columns = ['purpose']
purpose_data = pd.DataFrame(data = purpose, columns = columns) #создаю таблицу из списка
purpose_data['purpose_tag'] = '0' #прибавляю к таблице столбец с тегами целей кредита

#в цикле перебираю строки с целями кредита, получаю от них леммы и ключевые леммы записываю в теги
for i in range(len(purpose_data)): 
    lemm = m.lemmatize(purpose_data['purpose'][i])
    #print(lemm)
    
    if ('жилье' in lemm) or ('недвижимость' in lemm):
        purpose_data['purpose_tag'][i] = 'недвижимость'
    
    if 'автомобиль' in lemm:
        purpose_data['purpose_tag'][i] = 'автомобиль'
        
    if 'образование' in lemm:
        purpose_data['purpose_tag'][i] = 'образование'
        
    if 'свадьба' in lemm:
        purpose_data['purpose_tag'][i] = 'свадьба'
        
purpose_data[purpose_data['purpose_tag'] == '0'] #проверяю не осталось ли строчек без тега
data = data.merge(purpose_data, on='purpose', how='left') #объединяю общую таблицу и таблицу с тегами
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_tag
0,1,8437.0,42,высшее,0,женат / замужем,0,F,сотрудник,0,253876,покупка жилья,недвижимость
1,1,4024.0,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623.0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145886,покупка жилья,недвижимость
3,3,4124.0,32,среднее,1,женат / замужем,0,M,сотрудник,0,267629,дополнительное образование,образование
4,0,340266.0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


**Вывод**

В столбце с целями получения кредита были выделены ключевые слова (теги), по которым теперь можно объядинять получателей в группы

### Категоризация данных

In [6]:
def income_group(income): #функция категоризации по уровню дохода
    
    if income <= 130000: #присваивается низкий уровень
        return 'низкий'
    
    if 130000 < income <= 200000: #присваивается средний уровень
        return 'средний'
    
    if income > 200000: #присваивается высокий уровень
        return 'высокий'
    
data['income_group'] = data['total_income'].apply(income_group) #применяем функцию к столбцу
#с уровнем дохода, собираем новый столбец с группами по уровню дохода
data['income_group'].value_counts()

низкий     8103
средний    7945
высокий    5474
Name: income_group, dtype: int64

**Вывод**

Для ответа на вопросы достаточно разбить на категории столбец с уровнем дохода, хотя, мне не известно, в каких денежных единицах он указан. Границы групп выбраны так, чтобы количество людей в группах было примерно одинаковым

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
#объединяю группы с 3 и более детми в группу '>=3', так как они малочисленные
data.loc[data['children'] == 3, 'children'] = '>=3'
data.loc[data['children'] == 4, 'children'] = '>=3'
data.loc[data['children'] == 5, 'children'] = '>=3'
data['children'].value_counts() #проверяю результат
children_debt = data.groupby('children')['debt'].sum() #количество должников по кредиту в зависимости 
#от кол-ва детей. На мой взгляд не показательно, так как людей с более чем 3 детьми очень мало
total = data['children'].value_counts() #всего людей с определенным количеством детей в таблице
children_part = children_debt / total #отношение имевших кредит к общему количеству
children_part.sort_values()

children
0      0.075129
>=3    0.081579
1      0.091470
2      0.094791
dtype: float64

**Вывод**

Заёмщики без детей или с количеством детей 3 и более реже не возвращают кредиты, в отличии от заёмщиков с 1 или 2мя детьми

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [8]:
total = data['family_status'].value_counts() #всего людей в группах по семейному положению
family_debt = data.groupby('family_status')['debt'].sum() #кол-во должников в этих группах
family_part = family_debt / total #отношение, показывающее процент должников в каждой группе
family_part.sort_values()

вдовец / вдова           0.065625
в разводе                0.071130
женат / замужем          0.075202
гражданский брак         0.092890
Не женат / не замужем    0.097405
dtype: float64

**Вывод**

Холостяки и люди, состоящие в гражданском браке, чаще имеют задолженность по кредиту, чем женатые, разведенные или вдовцы. Это может быть связано с тем, что люди, побывавшие в браке более отвественные и финансово стабильные

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [9]:
total = data['income_group'].value_counts() #всего людей в группах по семейному положению
income_debt = data.groupby('income_group')['debt'].sum() #кол-во должников в этих группах
income_part = income_debt / total #отношение, показывающее процент должников в каждой группе
print(income_part.sort_values())
data['debt'].sum() / data['income_group'].count() #отношение должников к общему количеству
#среди всех заемщиков таблицы

высокий    0.069967
низкий     0.081081
средний    0.088232
dtype: float64


0.08089396896199239

**Вывод**

Результаты напрямую зависят от выбора границ, по которым доходы делятся на высокие, средние и низкие. От уровня дохода задержка выплаты не зависит.

Забавно, что после замены пропусков в столбце с доходом, стало заметно, что люди с высоким уровнем дохода выплачивают кредиты лучше, чем все остальные. Но, опять же, я думаю, что изменилось соотношение в группах по доходам.

- Как разные цели кредита влияют на его возврат в срок?

In [10]:
total = data['purpose_tag'].value_counts() #всего людей в группах по целям
purpose_debt = data.groupby('purpose_tag')['debt'].sum() #кол-во должников в этих группах
purpose_part = purpose_debt / total #отношение, показывающее процент должников в каждой группе
purpose_part.sort_values()

недвижимость    0.072140
свадьба         0.079216
образование     0.091994
автомобиль      0.093395
dtype: float64

**Вывод**

Меньше всего должников, среди бравших кредит на покупку недвижимости и на свадьбу. Больше среди тех, кто брал кредит на образование и покупку автомобиля. Возможно, потому что кредит на свадьбу обычно сравнительно небольшой, а из-за невыплаты по кредиту за недвижимость (которую многие покупают как жилье), человек может лишиться основного жилья, и отвественность очень высокая

## Шаг 4. Общий вывод

Анализ проведен на основе статистики о платёжеспособности клиентов, предоставленной банком.
В данных были обнаружены следующие аномалии: пропуски, дубликаты, неправильный тип данных, случайные опечатки.
В предобработку данных входило: заполнение пропущенных значений, замена вещественного типа данных на целочисленный, удаление дубликатов, лемматизация строковых данных и разбиение на категории.
Количество детей и семейное положение влияют на факт погашения кредита в срок: люди без детей и с тремя и более детьм возвращают кредиты чаще, чем люди с одним и двумя детьми. Так же люди в браке, разведенные и вдовцы чаще возвращают кредиты.
Уровень дохода на задержку выплаты кредит в значительной степени не влияет.
А цели кредита влияют: взявшие кредит на покупку недвижимости или садьбу, возвращают его чаще, чем купившие на него автмобиль или получившие образование.